In [1]:
# ===========================================
#  Modelo_LGBM_Sencillo_v2
#  Alternativa simple, rápida y estable
# ===========================================

!pip install -q lightgbm

import pandas as pd
import numpy as np
from google.colab import files
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

# ---------------------------
# 1. Subir archivos
# ---------------------------
print("📌 Sube train.csv")
files.upload()
print("📌 Sube test.csv")
files.upload()

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Detectar ID automáticamente
id_col = [c for c in test.columns if c.lower()=="id"]
id_col = id_col[0] if id_col else test.columns[0]

TARGET = "RENDIMIENTO_GLOBAL"

# ---------------------------
# 2. Preparar datos
# ---------------------------
y_map = {v:k for k,v in enumerate(train[TARGET].unique())}
inv_map = {k:v for v,k in y_map.items()}

train["target_num"] = train[TARGET].map(y_map)

X = train.drop(columns=[TARGET, "target_num"])
y = train["target_num"]

# Unir para encoding
full = pd.concat([X, test], ignore_index=True)

# Preprocesado simple
for c in full.columns:
    if full[c].dtype == "object":
        full[c] = full[c].fillna("MISSING").astype(str).factorize()[0]
    else:
        full[c] = full[c].fillna(full[c].median())

X = full.iloc[:len(train)]
test_proc = full.iloc[len(train):]

# ---------------------------
# 3. Split y entrenamiento LightGBM
# ---------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

params = {
    "objective": "multiclass",
    "num_class": len(inv_map),
    "metric": "multi_logloss",
    "learning_rate": 0.06,
    "num_leaves": 40,
    "seed": 42
}

dtr = lgb.Dataset(X_train, y_train)
dva = lgb.Dataset(X_val, y_val)

model = lgb.train(
    params,
    dtr,
    num_boost_round=300,
    valid_sets=[dva],
    callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
)

# ---------------------------
# 4. Predicciones finales
# ---------------------------
preds = model.predict(test_proc, num_iteration=model.best_iteration)
labels = np.argmax(preds, axis=1)
labels = [inv_map[i] for i in labels]

submission = pd.DataFrame({
    id_col: test[id_col],
    TARGET: labels
})

submission.to_csv("submission_simple_v2.csv", index=False)
print("\n✔ Archivo 'submission_simple_v2.csv' generado correctamente.")
files.download("submission_simple_v2.csv")


📌 Sube train.csv


Saving train.csv to train.csv
📌 Sube test.csv


Saving test.csv to test.csv
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051507 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1515
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 19
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.371993
[LightGBM] [Info] Start training from score -1.391216
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's multi_logloss: 1.20654

✔ Archivo 'submission_simple_v2.csv' generado correctamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>